In [1]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

2023-06-27 01:42:40.527685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 01:42:41.278647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Default GPU Device:/device:GPU:0


2023-06-27 01:42:42.272054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 01:42:42.300941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 01:42:42.302386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import sklearn as sk
from matplotlib import pyplot as plt 
from transformers import AutoTokenizer, AutoModel
from os import listdir
from os.path import isfile
import os
import pickle
import sys
import os.path
from os import path
from transformers import TFAutoModel
from transformers import BertTokenizer

import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.optimizers.legacy import Adam

codebert_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#!pip install transformers==4.20.0

SEED = 42

tf.random.set_seed(SEED)

use_function_metrics = True

In [3]:
# Read dataset
data = pd.read_excel('labeled_dataset.xlsx')

X_df_all = data[data["message"].notna()]
y_df_all = data['comment_group'].astype('str')


In [4]:
def getCodes():
    codes = []
    for comment, comment_id, line_no in zip(X["message"], X['comment_id'], X['line_number']):
        line_no = int(line_no)
        old_code = None

        old_folder = code_folder + comment_id + '/Old/'
        if(os.path.isdir(old_folder)):
            old_code = ""
            
            for filename in listdir(old_folder):
                code_lines = read_code_lines(old_folder, filename, line_no)
                lines = "\n".join(code_lines)
                old_code = old_code + lines
        else:
            print("no old")

        new_code = None
        new_folder = code_folder + comment_id + '/New/'
        if(os.path.isdir(new_folder)):
            new_code = ""
            for filename in listdir(new_folder):
                code_lines = read_code_lines(new_folder, filename, line_no)
                lines = "\n".join(code_lines)
                new_code = new_code + lines
        else:
            print("no new")

        codes.append((old_code, new_code))
    return codes

In [5]:
file_path = 'codes.pkl'
if os.path.exists(file_path):
    f = open(file_path, 'rb')
    codes = pickle.load(f)
else:
    codes = getCodes()
    f = open(file_path, 'wb')
    pickle.dump(codes, f)
    # sys.getsizeof(codes)
    len(pickle.dumps(codes))

In [6]:
# Drop unused columns
X_df_all.drop(['project', 'revision_id',
            'parent_revision','branch','file_name','reviewer_id',
            'author_id','Downloaded Patch.1', 'URL','change_id','request_id','line_number','patchset_number' ], axis=1, inplace=True)


/tmp/ipykernel_5000/3622005745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df_all.drop(['project', 'revision_id',


In [7]:
code_df = pd.DataFrame(codes, columns=['old_code', 'new_code'])

X =pd.concat([X_df_all["message"], code_df["old_code"]], axis="columns")
y = pd.get_dummies(X_df_all['comment_group'])

Message_numpy=X["message"].to_numpy()

pd.get_dummies(X_df_all['comment_group'])

,DISCUSS,DOCUMENTATION,FALSE POSITIVE,FUNCTION,REFACTORING
0,False,False,False,False,True
1,False,False,False,True,False
2,False,False,False,False,True
3,False,False,False,False,True
4,False,True,False,False,False
...,...,...,...,...,...
1823,True,False,False,False,False
1824,False,True,False,False,False
1825,False,False,False,False,True
1826,True,False,False,False,False


In [8]:
# Function for returning classification report
def metrics(model, test_x, y_test, batch_size = 8):
    y_preds = model.predict(test_x, batch_size = batch_size)
    true_class = tf.argmax( y_test, 1 )
    predicted_class = tf.argmax( y_preds, 1 )

    print(classification_report(true_class, predicted_class))
    cm = confusion_matrix(true_class, predicted_class)
    
    return classification_report(true_class, predicted_class, output_dict=True), cm


In [9]:
# Concatanating code and review comments (message)
X_3x = pd.concat([X, code_df["new_code"]], axis="columns")

In [10]:
def tokenize_text(tokenizer, sentence, total_len):
    tokens = tokenizer.encode_plus(sentence,max_length=total_len, truncation=True,add_special_tokens=True, padding="max_length")

    return tokens["input_ids"], tokens["attention_mask"]


In [11]:
total_token_slots = 512

comment_input_ids = []
comment_attention_masks = []
old_code_input_ids = []
old_code_attention_masks = []
new_code_input_ids = []
new_code_attention_masks = []
for cols, (comment, old_code, new_code) in X_3x.iterrows():
    comment = str(comment)
    old_code = str(old_code)
    new_code = str(new_code)

    #input_ids, attention_mask = tokenize_text(bert_tokenizer, comment, total_token_slots)
    input_ids, attention_mask = tokenize_text(codebert_tokenizer, comment, total_token_slots)
    comment_input_ids.append(input_ids)
    comment_attention_masks.append(attention_mask)

    input_ids, attention_mask = tokenize_text(codebert_tokenizer, old_code, total_token_slots)
    old_code_input_ids.append(input_ids)
    old_code_attention_masks.append(attention_mask)

    input_ids, attention_mask = tokenize_text(codebert_tokenizer, new_code, total_token_slots)
    new_code_input_ids.append(input_ids)
    new_code_attention_masks.append(attention_mask)

comment_input_ids           = np.array(comment_input_ids)
comment_attention_masks     = np.array(comment_attention_masks)
old_code_input_ids          = np.array(old_code_input_ids)
old_code_attention_masks    = np.array(old_code_attention_masks)
new_code_input_ids          = np.array(new_code_input_ids)
new_code_attention_masks    = np.array(new_code_attention_masks)



In [12]:
def read_metrics():
    metrics_df_path = 'code_attributes.csv'
    metrics_df = pd.read_csv(metrics_df_path)

    print("metrics columns are eqaul check:", metrics_df['folderName'].equals(X_df_all['comment_id']))
    
    return metrics_df

In [13]:
metrics_df = read_metrics()
#metrics_df

metrics columns are eqaul check: True


In [14]:
def add_chosen_metrics(metrics_df, chosen_float_metric_columns):
    metrics_df = metrics_df.merge(X_df_all[chosen_float_metric_columns],left_index =True, right_index=True)
    chosen_float_metric_columns = metrics_df.columns 
    metrics_df = metrics_df.astype(np.float32)
    return metrics_df, chosen_float_metric_columns
def drop_error_columns(debugColumns):
    metrics_df.drop(debugColumns, axis=1, inplace=True)

In [15]:
chosen_float_metric_columns =['cyclomatic_complexity','comment_loc']

metrics_df["error"].replace(np.nan, "ok", inplace=True)

debugColumns = [
        "folderName",
        "numOldFiles",
        "numNewFiles",
        "error",
        "isDupe"
    ]

drop_error_columns(debugColumns)
metrics_df, chosen_metrics_columns = add_chosen_metrics(metrics_df, chosen_float_metric_columns)

In [16]:
scaler = MinMaxScaler(clip = True)
scaler.fit(metrics_df)
metrics_df = scaler.transform(metrics_df)

In [17]:
def partition_input_dict(inputDict):
    returnDict = dict()
    returnDict["input_ids"] = inputDict["input_ids"][dataIndex]
    returnDict["token_type_ids"] = inputDict["token_type_ids"][dataIndex]
    returnDict["attention_mask"] = inputDict["attention_mask"][dataIndex]
    return returnDict

In [18]:
data_input = {"comment_input_ids": comment_input_ids ,
              "comment_attention_mask": comment_attention_masks,
                "code_input_ids": old_code_input_ids,
                "code_attention_mask": old_code_attention_masks,
             "metric" : metrics_df}

In [19]:
# Combining all attributes for modeling together
X_all = [comment_input_ids, comment_attention_masks,
            old_code_input_ids, old_code_attention_masks,
            metrics_df]

In [20]:
lstm_layer_1_name = 'lstm_layer_1'
lstm_layer_2_name = "lstm_layer_2"
bert_layer_name = 'tf_roberta_model'
def single_bert_layer_hidden_state(prefix):
    input_ids_dummy = tf.keras.layers.Input(shape=(total_token_slots,), name=(prefix+'input_ids'), dtype='int64')
    mask_dummy = tf.keras.layers.Input(shape=(total_token_slots,), name=(prefix+'attention_mask'), dtype='int64')
    codebert = TFAutoModel.from_pretrained("microsoft/codebert-base")
    
    embeddings_dummy = codebert(input_ids= input_ids_dummy, attention_mask=mask_dummy)[0]

    return input_ids_dummy,mask_dummy,codebert,embeddings_dummy


In [21]:

def get_bert_lstm_text_code():
    # two input layers, we ensure layer name variables match to dictionary keys in TF dataset
    comment_input_ids_dummy, comment_mask_dummy, comment_codebert, comment_embeddings_dummy = single_bert_layer_hidden_state("comment_")
    old_code_input_ids_dummy, old_code_mask_dummy, old_code_codebert, old_code_embeddings_dummy = single_bert_layer_hidden_state("code_")
    
    comment_x = tf.keras.layers.LSTM(50, dropout=0.3, recurrent_dropout=0.3, name=lstm_layer_1_name)(comment_embeddings_dummy)
    old_code_x = tf.keras.layers.LSTM(50, dropout=0.3, recurrent_dropout=0.3, name=lstm_layer_2_name)(old_code_embeddings_dummy)
    all_lstm = tf.keras.layers.Concatenate()([comment_x, old_code_x])
    
    # x = tf.keras.layers.Dense(1024, activation='relu')(embeddings_dummy)
    float_metrics_input_dummy = tf.keras.layers.Input(shape=(len(chosen_metrics_columns),), name='metric',dtype='float64')
    
    x_with_metrics = tf.keras.layers.Concatenate()([all_lstm, float_metrics_input_dummy])
    
    y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x_with_metrics)

    # initialize model
    model = tf.keras.Model(inputs=[ comment_input_ids_dummy, comment_mask_dummy,
                                   old_code_input_ids_dummy, old_code_mask_dummy,
                                    float_metrics_input_dummy], outputs=y)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy()
    adam_optimizer = Adam(learning_rate=1e-5)
    model.compile(loss=loss,optimizer=adam_optimizer,metrics=[acc])
    
    return model

In [22]:
def partition_input_dict(inputDict, dataIndex):
    returnDict = dict()
    returnDict["comment_input_ids"] = inputDict["comment_input_ids"][dataIndex]
    returnDict["comment_attention_mask"] = inputDict["comment_attention_mask"][dataIndex]
    returnDict["code_input_ids"] = inputDict["code_input_ids"][dataIndex]
    returnDict["code_attention_mask"] = inputDict["code_attention_mask"][dataIndex]
    returnDict["metric"] = inputDict["metric"][dataIndex]
    #instance_id =inputDict
    return returnDict


In [23]:
def generate_test_train_index( length, ratio):
    rand_array=np.arange(0,length,1)
    train, test =train_test_split(rand_array,test_size=ratio, shuffle=True )
    train=np.sort(train)
    test=np.sort(test)
    return train, test

In [24]:
y=y.values.tolist()

y=np.array(y)

In [25]:
def results_with_misclassification(model, test_x, y_test, test_message_comment, batch_size = 8):
    
    retro_output=[]
    
    y_preds = model.predict(test_x, batch_size = batch_size)
    true_class = tf.argmax( y_test, 1 )
    predicted_class = tf.argmax( y_preds, 1 )
    
    t_class=true_class.numpy()
    pred_class=predicted_class.numpy()
    
    for i in range(0,len(true_class)):
        if(t_class[i]!=pred_class[i]):
            retro_output.append([test_message_comment[i],t_class[i],pred_class[i]])
    
    cm = confusion_matrix(true_class, predicted_class)
    print(classification_report(true_class, predicted_class))
    
    return classification_report(true_class, predicted_class, output_dict=True),cm,retro_output

In [26]:
# Code for modeling and calculating model performance

precision = [0.0, 0.0, 0.0, 0.0, 0.0]
recall = [0.0, 0.0, 0.0, 0.0, 0.0]
f_score = [0.0, 0.0, 0.0, 0.0, 0.0]
accuracy = 0.0

rows, cols = (5, 5)
confusion = [[0.0]*cols]*rows
error_output=[]
count=1

random_folding = KFold(n_splits=10, shuffle=True, random_state=SEED)
for train_index, test_index in random_folding.split(y):
    
    print("Fold: ",count)
    
    train_input_set = partition_input_dict(data_input, train_index)
    test_input = partition_input_dict(data_input, test_index)
    train_output = y[train_index,]
    test_output = y[test_index,]
    
    test_message_comment=Message_numpy[test_index,]
    
    
    training_index, validation_index = generate_test_train_index(train_output.shape[0], 0.1)
    model_training_input = partition_input_dict(train_input_set, training_index)
    validation_input = partition_input_dict(train_input_set, validation_index)
    model_training_output = train_output[training_index]
    validation_output = train_output[validation_index]
    
    
    model = get_bert_lstm_text_code()
    #model.summary()
    callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=0,
        patience=2,
        verbose=1,
        mode="min",
        baseline=None,
        restore_best_weights=True,)
    
    history = model.fit(model_training_input,
                        model_training_output, 
                        batch_size=4,
                        epochs=4,
                        validation_data=(validation_input, validation_output),
                        callbacks=[callback],
                        )
    
    res,confusion_mat, error= results_with_misclassification(model, test_input, test_output, test_message_comment, batch_size = 8)
    error_output.append(error)
    
    confusion += confusion_mat
    
    precision[0] += res['0']['precision']
    precision[1] += res['1']['precision']
    precision[2] += res['2']['precision']
    precision[3] += res['3']['precision']
    precision[4] += res['4']['precision']
    
    recall[0] += res['0']['recall']
    recall[1] += res['1']['recall']
    recall[2] += res['2']['recall']
    recall[3] += res['3']['recall']
    recall[4] += res['4']['recall']
    
    f_score[0] += res['0']['f1-score']
    f_score[1] += res['1']['f1-score']
    f_score[2] += res['2']['f1-score']
    f_score[3] += res['3']['f1-score']
    f_score[4] += res['4']['f1-score']
    
    accuracy += res['accuracy']
    
    count=count+1

Fold:  1


2023-06-27 01:43:27.424833: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 01:43:27.425400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-27 01:43:27.425886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/4
370/370 [==============================] - 461s 1s/step - loss: 1.4535 - categorical_accuracy: 0.4014 - val_loss: 1.2616 - val_categorical_accuracy: 0.5333
Epoch 2/4
370/370 [==============================] - 444s 1s/step - loss: 1.1255 - categorical_accuracy: 0.5764 - val_loss: 1.1981 - val_categorical_accuracy: 0.5394
Epoch 3/4
370/370 [==============================] - 442s 1s/step - loss: 0.8591 - categorical_accuracy: 0.6919 - val_loss: 1.1005 - val_categorical_accuracy: 0.5455
Epoch 4/4
23/23 [==============================] - 13s 422ms/step
              precision    recall  f1-score   support

           0       0.61      0.57      0.59        40
           1       0.64      0.83      0.72        30
           2       0.50      0.19      0.28        21
           3       0.32      0.28      0.30        29
           4       0.60      0.70      0.65        63

    accuracy                           0.57       183
   macro avg       0.53      0.51      0.51       183
we

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
370/370 [==============================] - 455s 1s/step - loss: 1.4787 - categorical_accuracy: 0.3689 - val_loss: 1.2982 - val_categorical_accuracy: 0.5212
Epoch 2/4
370/370 [==============================] - 439s 1s/step - loss: 1.2333 - categorical_accuracy: 0.5236 - val_loss: 1.1304 - val_categorical_accuracy: 0.5636
Epoch 3/4
370/370 [==============================] - 439s 1s/step - loss: 1.0112 - categorical_accuracy: 0.6142 - val_loss: 1.0367 - val_categorical_accuracy: 0.6303
Epoch 4/4
23/23 [==============================] - 13s 415ms/step
              precision    recall  f1-score   support

           0       0.51      0.67      0.58        43
           1       0.64      0.78      0.71        46
           2       0.20      0.08      0.12        12
           3       0.35      0.39      0.37        18
           4       0.73      0.52      0.61        64

    accuracy                           0.58       183
   macro avg       0.49      0.49      0.48       183
we

...

Epoch 1/4
370/370 [==============================] - 456s 1s/step - loss: 1.4770 - categorical_accuracy: 0.3649 - val_loss: 1.3977 - val_categorical_accuracy: 0.5091
Epoch 2/4
370/370 [==============================] - 439s 1s/step - loss: 1.2510 - categorical_accuracy: 0.5203 - val_loss: 1.2709 - val_categorical_accuracy: 0.4667
Epoch 3/4
370/370 [==============================] - 440s 1s/step - loss: 1.0125 - categorical_accuracy: 0.6068 - val_loss: 1.1148 - val_categorical_accuracy: 0.5939
Epoch 4/4
23/23 [==============================] - 13s 416ms/step
              precision    recall  f1-score   support

           0       0.59      0.55      0.57        47
           1       0.71      0.80      0.75        40
           2       0.00      0.00      0.00        22
           3       0.44      0.30      0.36        27
           4       0.45      0.72      0.56        47

    accuracy                           0.55       183
   macro avg       0.44      0.47      0.45       183
we

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
371/371 [==============================] - 608s 2s/step - loss: 1.4166 - categorical_accuracy: 0.4031 - val_loss: 1.2991 - val_categorical_accuracy: 0.4545
Epoch 2/4
371/371 [==============================] - 585s 2s/step - loss: 1.1532 - categorical_accuracy: 0.5449 - val_loss: 1.2441 - val_categorical_accuracy: 0.5091
Epoch 3/4
371/371 [==============================] - 588s 2s/step - loss: 0.9299 - categorical_accuracy: 0.6462 - val_loss: 1.1565 - val_categorical_accuracy: 0.5455
Epoch 4/4
23/23 [==============================] - 13s 418ms/step
              precision    recall  f1-score   support

           0       0.62      0.76      0.68        41
           1       0.83      0.81      0.82        47
           2       0.50      0.18      0.26        17
           3       0.80      0.15      0.26        26
           4       0.56      0.82      0.67        51

    accuracy                           0.65       182
   macro avg       0.66      0.54      0.54       182
we

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
371/371 [==============================] - 614s 2s/step - loss: 1.4271 - categorical_accuracy: 0.3957 - val_loss: 1.2682 - val_categorical_accuracy: 0.4970
Epoch 2/4
371/371 [==============================] - 587s 2s/step - loss: 1.1976 - categorical_accuracy: 0.5409 - val_loss: 1.1102 - val_categorical_accuracy: 0.5697
Epoch 3/4
371/371 [==============================] - 589s 2s/step - loss: 0.9570 - categorical_accuracy: 0.6583 - val_loss: 1.0840 - val_categorical_accuracy: 0.6000
Epoch 4/4
23/23 [==============================] - 13s 421ms/step
              precision    recall  f1-score   support

           0       0.62      0.60      0.61        42
           1       0.60      0.76      0.67        38
           2       0.21      0.33      0.26        15
           3       0.50      0.43      0.46        28
           4       0.67      0.53      0.59        59

    accuracy                           0.56       182
   macro avg       0.52      0.53      0.52       182
we

In [27]:
# Code for error analysis
debug_file_name = "error-analysis"+".xlsx"
from  functools import reduce
debug_info = reduce(lambda x,y: x+y, error_output)

debug_df=pd.DataFrame(debug_info, columns=["comment","true_class","predicted"])
debug_df.to_excel(debug_file_name)

In [28]:
# Final performance values
print([x/10 for x in precision])
print([x/10 for x in recall])
print([x/10 for x in f_score])
print(accuracy/10)
print(confusion)

[0.5895060350359798, 0.708204568197881, 0.3522619047619048, 0.42690824534942184, 0.6079775625538881]
[0.6723116127927378, 0.7699216306322485, 0.18538057530704588, 0.28184143315177795, 0.6370820168962087]
[0.6251164325050699, 0.7339907898768459, 0.2101018834474166, 0.31658896871552394, 0.6106123883939973]
0.5875457875457875
[[300.  32.  20.  22.  71.]
 [ 30. 297.   8.   3.  49.]
 [ 45.  23.  28.  23.  39.]
 [ 37.  14.  18.  71. 100.]
 [101.  56.  15.  48. 378.]]
